In [ ]:
# 1. Установка необходимых библиотек
!pip install ultralytics

In [ ]:
# 2. Клонирование репозитория TACO
!git clone https://github.com/pedropro/TACO.git
%cd TACO

In [ ]:
# 3. Создание скрипта для быстрой загрузки изображений (многопоточный)
download_script = '''
import os.path
import argparse
import json
from PIL import Image
import requests
from io import BytesIO
import sys
import concurrent.futures
import threading

parser = argparse.ArgumentParser(description='')
parser.add_argument('--dataset_path', required=False, default= './data/annotations.json', help='Path to annotations')
parser.add_argument('--threads', required=False, type=int, default=20, help='Number of parallel downloads')
args = parser.parse_args()

dataset_dir = os.path.dirname(args.dataset_path)

print("Starting download...")

# Thread-safe counter for progress
counter = 0
counter_lock = threading.Lock()
nr_images = 0

def download_one_image(image):
    global counter
    
    file_name = image["file_name"]
    url_original = image["flickr_url"]
    url_resized = image["flickr_640_url"]

    file_path = os.path.join(dataset_dir, file_name)

    # Create subdir if necessary
    subdir = os.path.dirname(file_path)
    if not os.path.isdir(subdir):
        try:
            os.makedirs(subdir, exist_ok=True)
        except OSError:
            pass

    if not os.path.isfile(file_path):
        # Load and Save Image
        try:
            # Try original
            response = requests.get(url_original, timeout=15)
            if response.status_code == 200:
                img = Image.open(BytesIO(response.content))
                if hasattr(img, "_getexif") and img._getexif():
                    img.save(file_path, exif=img.info["exif"])
                else:
                    img.save(file_path)
            else:
                # Try resized if original fails
                response = requests.get(url_resized, timeout=15)
                if response.status_code == 200:
                    img = Image.open(BytesIO(response.content))
                    if hasattr(img, "_getexif") and img._getexif():
                        img.save(file_path, exif=img.info["exif"])
                    else:
                        img.save(file_path)
        except Exception as e:
            pass

    with counter_lock:
        counter += 1
        if counter % 50 == 0 or counter == nr_images:
            sys.stdout.write(f"\rDownloaded {counter}/{nr_images}")
            sys.stdout.flush()

# Load annotations
with open(args.dataset_path, 'r') as f:
    annotations = json.loads(f.read())
    images = annotations['images']
    nr_images = len(images)

    print(f"Downloading {nr_images} images using {args.threads} threads...")
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=args.threads) as executor:
        executor.map(download_one_image, images)

    print('\nFinished')
'''

with open('download_fast.py', 'w') as f:
    f.write(download_script)

In [ ]:
# 4. Запуск загрузки изображений
!python download_fast.py

In [ ]:
# 4.1 Создание карты классов (Basic 5: Plastic, Paper, Glass, Metal, Organic, Other)
map_basic_content = """Aerosol,Metal
Aluminium foil,Metal
Battery,Other
Aluminium blister pack,Other
Carded blister pack,Other
Clear plastic bottle,Plastic
Glass bottle,Glass
Other plastic bottle,Plastic
Plastic bottle cap,Plastic
Metal bottle cap,Metal
Broken glass,Glass
Drink can,Metal
Food Can,Metal
Corrugated carton,Paper
Drink carton,Paper
Egg carton,Paper
Meal carton,Paper
Other carton,Paper
Paper cup,Paper
Disposable plastic cup,Plastic
Foam cup,Plastic
Glass cup,Glass
Other plastic cup,Plastic
Food waste,Organic
Plastic lid,Plastic
Metal lid,Metal
Magazine paper,Paper
Tissues,Paper
Wrapping paper,Paper
Normal paper,Paper
Paper bag,Paper
Plastified paper bag,Paper
Pizza box,Paper
Garbage bag,Plastic
Single-use carrier bag,Plastic
Polypropylene bag,Plastic
Produce bag,Plastic
Cereal bag,Plastic
Bread bag,Plastic
Plastic film,Plastic
Crisp packet,Plastic
Other plastic wrapper,Plastic
Retort pouch,Plastic
Spread tub,Plastic
Tupperware,Plastic
Disposable food container,Plastic
Foam food container,Plastic
Other plastic container,Plastic
Plastic glooves,Plastic
Plastic utensils,Plastic
Pop tab,Metal
Rope & strings,Other
Scrap metal,Metal
Shoe,Other
Six pack rings,Plastic
Squeezable tube,Plastic
Plastic straw,Plastic
Paper straw,Paper
Styrofoam piece,Plastic
Toilet tube,Paper
Unlabeled litter,Other
Glass jar,Glass
Other plastic,Plastic
Cigarette,Other
"""

with open('detector/taco_config/map_basic.csv', 'w') as f:
    f.write(map_basic_content)

In [ ]:
# 5. Создание скрипта конвертации в YOLO
convert_script = '''
import json
import os
import shutil
import csv
import random
import yaml
from tqdm import tqdm

# Configuration
ANNOTATIONS_FILE = 'data/annotations.json'
IMAGES_DIR = 'data'
OUTPUT_DIR = 'yolo_dataset'
MAP_FILE = 'detector/taco_config/map_basic.csv'

def load_class_map(map_file):
    mapping = {}
    new_classes = set()
    with open(map_file, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            if len(row) >= 2:
                mapping[row[0]] = row[1]
                new_classes.add(row[1])
    
    # Sort classes to ensure consistent ordering
    sorted_classes = sorted(list(new_classes))
    class_to_id = {name: i for i, name in enumerate(sorted_classes)}
    
    return mapping, class_to_id, sorted_classes

def convert_bbox(size, box):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = box[0] + box[2] / 2.0
    y = box[1] + box[3] / 2.0
    w = box[2]
    h = box[3]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return (x, y, w, h)

def main():
    # 1. Load Class Mapping
    print("Loading class mapping...")
    cat_name_map, new_class_to_id, class_names = load_class_map(MAP_FILE)
    print(f"Found {len(class_names)} target classes: {class_names}")

    # 2. Load Annotations
    print("Loading annotations...")
    with open(ANNOTATIONS_FILE, 'r') as f:
        data = json.load(f)

    # Create ID to Name mapping for original categories
    original_cat_id_to_name = {cat['id']: cat['name'] for cat in data['categories']}

    # 3. Prepare Directories
    for split in ['train', 'val']:
        os.makedirs(os.path.join(OUTPUT_DIR, 'images', split), exist_ok=True)
        os.makedirs(os.path.join(OUTPUT_DIR, 'labels', split), exist_ok=True)

    # 4. Process Images
    images = data['images']
    annotations = data['annotations']
    
    # Group annotations by image_id
    img_anns = {}
    for ann in annotations:
        img_id = ann['image_id']
        if img_id not in img_anns:
            img_anns[img_id] = []
        img_anns[img_id].append(ann)

    # Split dataset
    random.seed(42)
    random.shuffle(images)
    split_idx = int(len(images) * 0.9)
    train_images = images[:split_idx]
    val_images = images[split_idx:]

    print("Converting data...")
    
    for split, split_images in [('train', train_images), ('val', val_images)]:
        for img in tqdm(split_images, desc=f"Processing {split}"):
            img_id = img['id']
            file_name = img['file_name']
            
            # Source path
            src_path = os.path.join(IMAGES_DIR, file_name)
            if not os.path.exists(src_path):
                continue

            # Destination path
            # Flatten directory structure for YOLO (batch_1/001.jpg -> batch_1_001.jpg)
            flat_name = file_name.replace('/', '_').replace('\\', '_')
            dst_img_path = os.path.join(OUTPUT_DIR, 'images', split, flat_name)
            dst_label_path = os.path.join(OUTPUT_DIR, 'labels', split, flat_name.replace('.jpg', '.txt'))

            # Copy image
            shutil.copy(src_path, dst_img_path)

            # Create label file
            with open(dst_label_path, 'w') as out_f:
                if img_id in img_anns:
                    for ann in img_anns[img_id]:
                        original_cat_id = ann['category_id']
                        original_name = original_cat_id_to_name.get(original_cat_id)
                        
                        # Map to new class
                        if original_name in cat_name_map:
                            new_class_name = cat_name_map[original_name]
                            new_class_id = new_class_to_id[new_class_name]
                            
                            # Convert bbox
                            bbox = convert_bbox((img['width'], img['height']), ann['bbox'])
                            
                            out_f.write(f"{new_class_id} {bbox[0]:.6f} {bbox[1]:.6f} {bbox[2]:.6f} {bbox[3]:.6f}\\n")

    # 5. Create data.yaml
    yaml_content = {
        'path': os.path.abspath(OUTPUT_DIR),
        'train': 'images/train',
        'val': 'images/val',
        'names': {i: name for i, name in enumerate(class_names)}
    }
    
    with open(os.path.join(OUTPUT_DIR, 'data.yaml'), 'w') as f:
        yaml.dump(yaml_content, f, sort_keys=False)

    print(f"Conversion complete! Dataset saved to {OUTPUT_DIR}")

if __name__ == '__main__':
    main()
'''

with open('convert_to_yolo.py', 'w') as f:
    f.write(convert_script)

In [ ]:
# 6. Запуск конвертации
!python convert_to_yolo.py

In [ ]:
# 7. Обучение модели YOLOv8
from ultralytics import YOLO
import torch

# Автоматическое определение устройства (GPU или CPU)
device = 0 if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

if device == 'cpu':
    print("⚠️ ВНИМАНИЕ: GPU не найден! Обучение будет очень медленным.")
    print("Если вы в Google Colab: Зайдите в 'Среда выполнения' -> 'Сменить среду выполнения' -> выберите 'T4 GPU'.")

# Используем модель Small (s) для баланса скорости и точности
model = YOLO('yolov8s.pt')

results = model.train(
    data='yolo_dataset/data.yaml',
    epochs=50,
    imgsz=640,
    batch=16,
    name='taco_yolov8',
    device=device
)

In [ ]:
# 8. Архивация результатов для скачивания
!zip -r taco_yolov8_results.zip runs/detect/taco_yolov8

In [ ]:
# 9. Скачивание архива (если не скачалось автоматически, найдите файл в панели слева)
from google.colab import files
files.download('taco_yolov8_results.zip')